In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# создание сессии с помощью SparkSession

spark = SparkSession.builder.appName('Practice').getOrCreate()

spark.sql("select 'hello spark'").collect()[0][0]

In [ ]:
# создание ссесии с помощью SparkContext

from pyspark import SparkContext, SparkConf

conf = SparkConf().setMaster("local").setAppName("newSession")

sc = SparkContext(conf=conf)

spark.sql("select 'hello spark'").collect()[0][0]

In [ ]:
spark

In [ ]:
# конвертирую excel в csv

import pandas as pd

df_excel = pd.read_excel('cvm_srv.xlsx')

df_excel.to_csv('cvm_srv.csv', index=False)

In [ ]:
# чтение фрейма данных

# InferSchema - автоматическое определение типов данных при чтении файла

df_spark = spark.read.option('header', 'true').csv('cvm_srv.csv', inferSchema=True)

In [ ]:
# вывод результата

df_spark.show(1)

In [ ]:
# вывод схемы

df_spark.printSchema()

In [ ]:
# получение колонок

df_spark.columns

In [ ]:
# получение кортежа из строк
df_spark.head(1)[0]

In [ ]:
# Выборка по полю

df_spark.select('regid', 'dayy').show()

# df_spark['regid', 'dayy'].show()

#df.regid

In [ ]:
# Добавление поля

df_spark = df_spark.withColumn("new_column", df_spark['regid']+1)

In [ ]:
# Удаление поля

df_spark = df_spark.drop('new_column')

In [ ]:
# Переименование поля

df_spark.withColumnRenamed("regid", "regid_new")

In [ ]:
# удаление строк с пустыми полями

# drop (how = как удалять, subset = из каких полей, thresh = какая то мера удаления)

df_spark.na.drop()

In [ ]:
# чем заполнить пробелы (Null)

df_spark.na.fill('missing_values',['name_cvm'])

In [ ]:
# заполнить пробелы средним значением

# можно выбрать также mod, median

from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols = ["regid","dayy","new_column"],
                 outputCols = ["{}_imputed".format(c) for c in ["regid","dayy","new_column"]]).setStrategy("mean") 

imputer.fit(df_spark).transform(df_spark).show()

In [ ]:
# фильтры

df_spark.filter("regid=68").show()

df_spark.filter(df_spark['regid']>68).show()

In [ ]:

# фильтры с условием И (&) и ИЛИ (|)

df_spark.filter((df_spark['regid']>68) & (df_spark['dayy'] == 70)).show()

df_spark.filter((df_spark['regid']>68) | (df_spark['dayy'] == 70)).show()

In [ ]:
# фильтры с условием НЕ (~)

df_spark.filter(~(df_spark['regid']>68)).show()

In [ ]:
# Фильтр like

df_spark.filter(df_spark['global_code'].like('PE5%')).show()

In [ ]:
# создание DataFrame (самостоятельно)

data = [('Lucy', 10, 3_000),('Tanya', 35, 200_000), ('Kolya', 15, 0)]

df = spark.createDataFrame(data, ['name', 'age', 'money'])


In [ ]:
# Группировка

df_spark.printSchema()

In [ ]:
# Группировка
# у группировки много есть агрегирующих функций (sum, min, max и тд.) Если не указывать ничего внутри функции, 
# он будет суммировать все поля числовые, если указать внутри функции, то только их

df_spark.groupBy('global_code').sum('service', 'dayy')

# df_spark.groupBy('global_code').sum()

In [ ]:
# среднее и подсчет элементов

df_spark.groupBy('global_code').mean().show()

df_spark.groupBy('global_code').count().show()

In [ ]:
# агрегирующая функция (два варианта применения)
from pyspark.sql.functions import sum

df_spark.groupBy("global_code").agg({'regid':'sum'}).show()

df_spark.groupBy("global_code").agg(sum('regid').alias('cnt_day')).show()

In [ ]:
# сортировка 


# df_spark.sort(df_spark['dayy'].asc()).show()

df_spark.orderBy(df_spark['dayy'].asc()).show()
